In [ ]:
import random
import string

# Function to generate a seed from the key
def generate_seed(key):
    return sum((ord(char.lower()) - ord('a') + 1) for char in key if char.isalpha())

# Function to generate a 55x55 matrix using the seed
def generate_matrix(seed):
    chars = list(string.ascii_lowercase + string.ascii_uppercase + ' ,.')
    random.seed(seed)
    matrix = []
    for _ in range(55):
        random.shuffle(chars)
        matrix.append(chars.copy())
    return matrix

# Function to shift letters up by one
def shift_up(key):
    shifted_key = ''
    for char in key:
        if char.islower():
            shifted_key += chr((ord(char) - ord('a') + 1) % 26 + ord('a'))
        elif char.isupper():
            shifted_key += chr((ord(char) - ord('A') + 1) % 26 + ord('A'))
        else:
            shifted_key += char
    return shifted_key

# Function to generate the new key
def generate_new_key(key):
    numerical_value = generate_seed(key)
    n = numerical_value % 26
    new_char = chr(ord('a') + n)
    new_key = key + new_char
    shifted_key = shift_up(new_key)
    return shifted_key

# Function to encrypt the plaintext using the key
def encrypt(plaintext, key):
    seed = generate_seed(key)
    matrix = generate_matrix(seed)
    ciphertext = []
    
    key_indices = [ord(char.lower()) - ord('a') for char in key if char.isalpha()]
    key_length = len(key_indices)
    
    for i, char in enumerate(plaintext):
        if char in string.ascii_lowercase + string.ascii_uppercase + ' ,.':
            row = key_indices[i % key_length]
            if char.islower():
                col = string.ascii_lowercase.index(char)
            elif char.isupper():
                col = 26 + string.ascii_uppercase.index(char)
            else:
                col = 52 + ' ,.'.index(char)
            ciphertext.append(matrix[row][col])
        else:
            ciphertext.append(char)  # Preserve characters that are not in the matrix
    
    return ''.join(ciphertext)[::-1]  # Reverse the ciphertext before returning

# Function to decrypt the ciphertext using the key
def decrypt(ciphertext, key):
    seed = generate_seed(key)
    matrix = generate_matrix(seed)
    plaintext = []
    
    key_indices = [ord(char.lower()) - ord('a') for char in key if char.isalpha()]
    key_length = len(key_indices)
    
    ciphertext = ciphertext[::-1]  # Reverse the ciphertext back to original order
    
    for i, char in enumerate(ciphertext):
        if char in string.ascii_lowercase + string.ascii_uppercase + ' ,.':
            row = key_indices[i % key_length]
            col = matrix[row].index(char)
            if col < 26:
                original_char = string.ascii_lowercase[col]
            elif col < 52:
                original_char = string.ascii_uppercase[col - 26]
            else:
                original_char = ' ,.'[col - 52]
            plaintext.append(original_char)
        else:
            plaintext.append(char)  # Preserve characters that are not in the matrix
    
    return ''.join(plaintext)

# Main function to get input and perform encryption and decryption
def main():
    #layers = 10
    while True:
        choice = input("Would you like to (e)ncrypt, (d)ecrypt, or (q)uit? ")
        if choice.lower() == 'e':
            plaintext = input("Enter the plaintext: ")
            key = input("Enter the key: ")
            layers = round((generate_seed(key))**0.5)
            encrypted_text = plaintext
            for _ in range(layers):
                encrypted_text = encrypt(encrypted_text, key)
                key = generate_new_key(key)
            print("Encrypted text:", encrypted_text)
        elif choice.lower() == 'd':
            ciphertext = input("Enter the ciphertext: ")
            key = input("Enter the key: ")
            keys = [key]
            for _ in range(layers - 1):
                key = generate_new_key(key)
                keys.append(key)
            decrypted_text = ciphertext
            for key in reversed(keys):
                decrypted_text = decrypt(decrypted_text, key)
            print("Decrypted text:", decrypted_text)
        elif choice.lower() == 'q':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 'e' to encrypt, 'd' to decrypt, or 'q' to quit.")

if __name__ == "__main__":
    main()
